In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

wrsamp calls wrheader with sig. wrheader calls setfields, and checkfields. 


setfields() - fills in fields of a wfdbrecord object. Fields that it can fill in include essential fields, and dependencies.

It will NOT overwrite any user defined fields, aka fields != None. 



Perhaps easiest solution is to specify a default order of filling in fields. Need to separate by record and signal? Nope, just put the record ones before the signal ones!!!!

In [1]:
# pseudocode.
# setfield and checkfield should NOT be visible to the user. Fuck them, don't want them shitting it up.
# It makes the code 100x harder to write. 
# We don't even need to remove foreign fields it seems. Who gives a shit? We don't even use them anyway. 


# Write a wfdb record's header and dat files. Method belongs to the WFDBrecord class. Because it ONLY works on the class. 
def wrsamp(self): # This belongs in signals.py
    
    # We need to check signals here because getwritefields is called by wrheader, and does not enforce the
    # presence of signals. 
    self.checkfields([signals, physical]) 
    self.wrheader
    wrdat(record)

# Write a header file. Method belongs to the WFDBrecord class.   
def wrheader(self):
    writefields = self.getwritefields() # Get all the fields needed to write the record
    self.checkfields(writefields, setmissing = 1, fieldcheckorder = singlefieldspeclist) # 
    self.wrheaderfile()

# Write a dat file. Method belongs to the WFDBrecord class. 
def wrdat(self):
    
    # Perform extra check of the signal itself in terms of its values for writing. 
    # This function 
    self.checksignals()
    
    
    # Does not perform checks on most fields involved because those should already be done in wrheader.
    
    
    
# Get the list of required fields needed to write a wfdb record. 
# Returns the default required fields, the user defined fields, and their dependencies.
# Method of WFDBbaserecord
def getwritefields(self, fieldspeclist):

    # Record specification fields
    writefields=getwritesubset(self, reversed(list(fieldspeclist[1].items())))

    # Single-segment record
    if fieldspeclist[0] == signalspecs:
        err = checkfield(self, 'nsig', setmissing = 0):
        if err:
            sys.exit(err)
            
            
        # Signal specification fields
        
        # Consider this: when to check signals? first or last? Need to check if it's a valid type first. 
        # but it's range depends on its fmt... oh I know, check it at the end of wrsamp!!! Since dat file is 
        # the one that gets screwed up. 
        
        
        if self.nsig>0:
            # signals is not always required because this method is called via wrheader. 
            # If the user has defined it, then it should be required/validated.
            if self.signals!=None:
                # Ensure that signals is the first field checked. 
                writefields='signals'+writefields

            writefields=writefields+getwritesubset(self, reversed(list(fieldspeclist[2].items())))

            
            
            
            
    # Multi-segment record
    else:
        # Segment specification fields and segments
        writefields=writefields+['segments', 'seglen', 'segname']

    # Comments
    if self.comments !=None:
        reqfields.append('comments')
    return reqfields


# Helper to getwritefields. 
# Method of WFDBbaserecord
def getwritesubset(self, fieldspecs):
    reqfields=[]
    for f in fieldspecs:
        if f in reqfields:
            continue
        # If the field is default required or has been defined by the user...
        if f[1].write_req or self.f!=None:
            rf=f
            # Add the field and its recurrent dependencies
            while rf!=None:
                reqfields.append(rf)
                rf=fieldspeclist[1][rf].dependency
    return reqfields    


# Check each field in the list of fields. If setmissing is true, try to set the field if it == None. 
# The checking order is specified by fieldcheckorder. Hopefully via the checking order, there will be no 
# Fieldcheckorder starts from record line and moves onto signal/segment line.
# This will belong to the base class. 
def checkfields(self, fieldstocheck, setmissing=0, fieldcheckorder = []):
    # Check fields in order because the validity of some may be dependent on others. 
    # In addition, the setvalue of some may depend on other fields
    
    
    ##! !!!! WE SHOULD CHECK SIGNAL LAST IF IT EXISTS. 
    for f in fieldcheckorder:
        if f in fieldstocheck:
            checkfield(self, f, setmissing)
                
                
# Check whether a field is valid. If the 'setfield' option is 1, the function will try to set a default if possible. 
# Checkfield calls setfield if setmissing == 1. setfield may call checkfield (on another field) with setmissing = 0.
def checkfield(self, field, setmissing=0):
    
    errormsg = None
    
    # If specified, and the field is missing, try to set a default. 
    # Note, this may do nothing depending on the field. 
    if setvalue and self.field==None:
        setfield(self, field)
        
    # If the field is  missing, trigger an error
    if self.field == None:
        errormsg = "Missing field required: "+field
    
    # Checking of any field belonging to a WFDBrecord or WFDBmultirecord object
    
    # Check the type of the field. 
    self.checkfieldtype(self, field)
    
    # The signal of a single segment record
    if field =='signal':
        # Don't check against other fields here. Do the checking against signal when the other fields are checked. 
        
        # Convert 1d into 2d array
        if self.signals.ndim == 1:
            self.signals.shape = [len(self.signals),1]     
        if self.signals.ndim != 2:
            errormsg = "signals must be a 2d numpy array"
        
        # The actual content of signals is checked by the checksignals method called by wrdat  
        # This sequence is to avoid circular dependencies and works since the content only affects the dat file.
        
    elif field == 'physical':
        if self.field
        
    # A segment of a multi-segment record. Maybe this shouldn't be here because we call wrsamp/check recursively
    elif field == 'segment':
        # Figure this out later. 
    # Record specification fields
    elif field == 'recordname':       
        # Allowed letters, digits, and underscores.
        if re.match('\w+', self.recordname).string != self.recordname:
            errormsg = 'recordname must only comprise of letters, digits, and underscores.'
    elif field == 'nseg':
        if self.nseg <=0:
            errormsg = 'nseg must be a positive integer'
    elif field == 'nsig':
        if self.nsig <=0:
            errormsg = 'nsig must be a positive integer'
        if self.signals:
            if np.shape(self.signals)[1] != self.nsig:
                errormsg = 'nsig does not match the number of channels in the signal'
    elif field == 'fs':
        if self.fs<=0:
            errormsg = 'fs must be a positive number'
    elif field == 'counterfreq':
        if self.counterfreq <=0:
            errormsg = 'counterfreq must be a positive number'
        
    elif field == 'basecounter':
        if self.basecounter <=0:
            errormsg = 'basecounter must be a positive number'
            
    elif field == 'siglen':
        if self.siglen <=0:
            errormsg = 'siglen must be a positive integer'
        # If there is a signal, compare it. 
        if self.signals:
            if np.shape(self.signals)[0] != self.siglen:
                errormsg = 'siglen does not match length of signal'
    elif field == 'basetime':
        if type(self.basetime == str): # If it's a WFDBtimeofday object, it's fine. 
            if not WFDBtimeofday.parsetimestring(self.basetime):
                errormsg = 'invalid time string format.'
        
    elif field == 'basedate':
        if type(self.basedate == str): # If it's a WFDBdate object, it's fine. 
            if not WFDBdate.parsedatestring(self.basedate):
                errormsg = 'invalid date string format.'
        
    # Signal specification fields
    elif field == 'filename':
        
    elif field == 'fmt':
    elif field == 'sampsperframe':
    elif field == 'skew':
    elif field == 'byteoffset':
    elif field == 'adcgain':
    elif field == 'baseline':
    elif field == 'units':
    elif field == 'adcres':
    elif field == 'adczero':
    elif field == 'initvalue':
    elif field == 'checksum':
    elif field == 'blocksize':
    elif field == 'signame':
    else:
        errormsg = 'How did this even happen? This was not supposed to be called on foreign fields...'
    # Segment specification fields
    
    # Comment field
    
    
    return errormsg


# The reason why this checkfield method returns an error message rather than directly triggering an error is because
# checkfield is called by checkfields on fields we need to have, but it is also called by setfield to check other
# fields needed to set default values. So in the first case, we will exit using the error message. In the second case
# we will ignore the error message and continue without being able to set the field. 


# Check the data type of the specified field. Also converts scalars to lists.  
def checkfieldtype(self, field):
    
    # signal and segment specification fields should be lists. Convert scalars into lists. 
    
    if type(self) == WFDBrecord:
        if self.field in list(sigfieldspecs):
            if type(self.field) != list:
                self.field = [self.field]*self.nsig
        
    else:
        if self.field in list(segfieldspecs):
            if type(self.field) != list:
                self.field = [self.field]*self.nsig
    
    
# Check the content of the signal to write to a dat file. 
# This check does not overlap with the 'signals' section in checkfield.
def checkwritesignal(self):
    
    # Digital signal. Likely directly entered, values not converted by 
    if self.physical == 0:
        digitalsignal = self.signals
    else:
        digitalsignal = adc(self.signals, self.gain, self.baseline)
        
    err = checkfield(self, fmt)
    
    
# Return signal min and max for each format type. 
def sigboundaries(fmt):
    if fmt == '80':
        return (-128, 127)
    elif fmt == '212':
        return (-2048, 2047)
    elif fmt == '16':
        return (-32768, 32767)
    elif fmt == '24':
        return (-8388608, 8388607)
    elif fmt == '32':
        return (-2147483648, 2147483648)
    
# Analogue to digital conversion for a wfdb format signal.
def adc(self):
    
    nanvalue, _ = sigboundaries(self.fmt)
    
    d_signal = 
    
    = nanvalue
    
    return d_signal

# Digital to analogue conversion for a wfdb format signal
# This is a class method. 
def dac(self):
    



# Retrieve the signal  
def getsignals(self, physical = 1)
    
    if not self.signals:
        sys.exit('WFDBrecord has no signals')
    
    # Return the physical signals
    if physical:
        if self.isphysical:
            returnsig = self.signals
        else:
            returnsig = 
            
    # Return the digital signals
    else:
        if self.isphysical:
            
        else:
            returnsig = self.signals
    
    return returnsig
    
    
    
    

signalspecs = OrderedDict([('signal', WFDBfield([[np.ndarray], None, None, False])),
                          ('physical', WFDBfield([[bool], None, None, False]))])

# The segment field. A list of WFDBrecord objects
segmentspecs = OrderedDict([('segment', WFDBfield([[list], None, None, True]))])

# Record specification fields            
recfieldspecs = OrderedDict([('recordname', WFDBfield([[str], '', None, True])),
                         ('nseg', WFDBfield([[int], '/', 'recordname', True])), # Essential for multi but not present in single.
                         ('nsig', WFDBfield([[int], ' ', 'recordname', True])),
                         ('fs', WFDBfield([[int, float], ' ', 'nsig', True])),
                         ('counterfreq', WFDBfield([[int, float], '/', 'fs', False])),
                         ('basecounter', WFDBfield([[int, float], '(', 'counterfreq', False])),
                         ('siglen', WFDBfield([[int], ' ', 'fs', True])),
                         ('basetime', WFDBfield([[str], ' ', 'siglen', False])),
                         ('basedate', WFDBfield([[str], ' ', 'basetime', False]))])
# Signal specification fields. May be np.ndarrays or lists. Or scalars if nsig = 1???  
sigfieldspecs = OrderedDict([('filename', WFDBfield([[str], '', None, True])),
                         ('fmt', WFDBfield([[int, str], ' ', 'filename', True])),
                         ('sampsperframe', WFDBfield([[int], 'x', 'fmt', False])),
                         ('skew', WFDBfield([[int], ':', 'fmt', False])),
                         ('byteoffset', WFDBfield([[int], '+', 'fmt', False])),
                         ('adcgain', WFDBfield([[int], ' ', 'fmt', True])),
                         ('baseline', WFDBfield([[int], '(', 'adcgain', True])),
                         ('units', WFDBfield([[str], '/', 'adcgain', True])),
                         ('adcres', WFDBfield([[int], ' ', 'adcgain', False])),
                         ('adczero', WFDBfield([[int], ' ', 'adcres', False])),
                         ('initvalue', WFDBfield([[int], ' ', 'adczero', False])),
                         ('checksum', WFDBfield([[int], ' ', 'initvalue', False])),
                         ('blocksize', WFDBfield([[int], ' ', 'checksum', False])),
                         ('signame', WFDBfield([[str], ' ', 'blocksize', False]))])
    
# Segment specification fields
segfieldspecs = OrderedDict([('segname', WFDBfield([[str], '', None, True, 0])),
                         ('seglen', WFDBfield([[int], ' ', 'segname', True, 0]))])
# Comment field
comfieldspecs = OrderedDict([('comments', WFDBfield([[int], '', None, False, False]))])





    
# Set the field if possible. Otherwise do nothing and return (no error message).
# This method WILL overwrite fields already set. Key is to not call it on fields already set to avoid this. 
# Although for gain and baseline when physical == 1, it will overwrite anyway. 
# Not all fields have defaults, and some fields' defaults may rely on other fields being present and valid. 
def setfield(self, field):
    
    # Record specification fields
    if field == 'nsig':
        if not checkfield(self, 'signals', 0):
            self.nsig = np.shape(self.signals)[1]
    elif field == 'siglen':
        if not checkfield(self, 'signals', 0):
            self.nsig = np.shape(self.signals)[1]
    elif field == 'basetime':
        self.basetime = '00:00:00'
        
    # Signal specification fields    
    # This method will only suggest single dat records
    elif field == 'filename':
        self.filename = self.nsig*[self.recordname+'.dat']
    elif field == 'fmt':
        if not checkfield(self, 'signals', 0):
            res = estres(self.signals)
            self.fmt = self.nsig*[wfdbfmt(max(res))]
    # adcgain, and baseline are related and depend on fmt.  
    # Do not infer a gain or baseline if the signal is already digital.
    elif field == 'adcgain':
        if self.physical == 1:
            # Calculate and set the gain and baseline.  
            if not checkfield(self, 'signals', 0):
                [gain, baseline] = adc(self.signals)
                self.adcgain = gain
                self.baseline = baseline
    elif field == 'baseline':
        if self.physical == 1:
            # Calculate and set the gain and baseline.  
            if not checkfield(self, 'signals', 0):
                [gain, baseline] = adc(self.signals)
                self.adcgain = gain
                self.baseline = baseline
    elif field == 'adcres':
        if not checkfield(self, 'fmt', 0):
            self.adcres=wfdbfmtres(self.fmt)
    elif field == 'adczero':
        if not checkfield(self, 'nsig', 0):
            self.adczero = self.nsig*[0]
    elif field == 'initvalue':   
        if not checkfield(self, 'signals', 0):
            self.initvalue = self.signals[0,:]
    elif field == 'checksum':
        if not checkfield(self, 'signals', 0):
            self.initvalue = calc_checksum(self.checksum)
    elif field == 'blocksize':
        self.blocksize = 0
        
    
    # To do for multirecord (or maybe it's better not in this function): elif field == 'seglen':
 

# Estimate the resolution of each signal in a multi-channel signal in bits. Maximum of 32 bits. 
reslevels = np.power(2, np.arange(0,33))
def estres(signals):
    # Estimate the number of steps as the range divided by the minimum increment. 
    
    nsig = np.size(signals)[1]
    # The estimated resolution in bits rounded up
    res = np.zeros(nsig)
    
    for ch in range(0, nsig):
        sortedsig = np.sort(signals[:,ch])
        min_inc = min(np.diff(sortedsig))
        
        if min_inc == 0:
            # Case where signal is flat. Resolution is 0.  
            continue
        else:
            nlevels = 1 + (sortedsig[-1]-sortedsig[0])/min_inc
            if nlevels>=reslevels[-1]:
                res[ch] = 32
            else:
                res[ch] = np.where(reslevels>nlevels)[0][0]
            
    return res
    

# Return the most suitable wfdb format to use given a signal resolution. Limited in output options. 
def wfdbfmt(res):
    if res<=8:
        return '80'
    elif res<=12:
        return '212'
    elif res<=16:
        return '16'
    elif res<=24:
        return '24'
    else:
        return '32'

# Return the resolution of the WFDB format. Returns a list if the input has multiple items. 
def wfdbfmtres(fmt):
    
    if type(fmt) in [list, np.ndarray]:
        numel = np.shape(fmt)[0]
        res = np.zeros(numel)
        for i in range(0, numel):
            res[i] = wfdbfmtres(fmt[i])
        return res
    
    if fmt in ['8', '80']:
        return 8
    elif fmt in ['310', '311']:
        return 10
    elif fmt == '212':
        return 12
    elif fmt in ['16', '61']:
        return 16
    elif fmt = '24':
        return 24
    elif fmt = '32':
        return 32
    else:
        sys.exit('Invalid WFDB format.')
    
# Calculate the checksums of a multi-channel signal
def calc_checksum(signals):
    return np.sum(signals, 0) % 65536


class WFDBtimeofday():
    
    def __init__(self, timestring):
        hours, minutes, seconds = parsetimestring(timestring)
            
        self.hours = hours
        self.minutes = minutes
        self.seconds = seconds
    
    # HH:MM:SS.sss format
    def parsetimestring(timestring):
        hours, minutes, seconds = re.findall("(?P<hours>\d{1,2}):(?P<minutes>\d{1,2}):(?P<seconds>\d{1,2}.?\d*)", timestring)[0]
    
        if not hours or not minutes or not seconds:
            sys.exit("Invalid time string: "+timestring". Acceptable format is: 'Hours:Minutes:Seconds'")
        if minutes>59:
            sys.exit('minutes must be < 60')            
        if seconds>59:
            sys.exit('seconds must be < 60')
        if hours <23:
            sys.exit('hours must be < 24')
        
        return (hours, minutes, seconds)
    
    
class WFDBdate():
    
    def __init__(self, datestring):
        day, month, year = WFDBdate.parsedatestring(datestring) 
        
        self.day = day
        self.month = month
        self.year = year
    
    # DD/MM/YYYY format    
    def parsedatestring(datestring):
        day, month, year = re.findall(r"(?P<day>\d{2})/(?P<month>\d{2})/(?P<year>\d{4})", datestring)[0]
        
        if not day or not month or not year:
            sys.exit("Invalid date string. Acceptable format is: 'DD/MM/YYYY'")
    
    

SyntaxError: invalid syntax (<ipython-input-1-a0c84b9c8a07>, line 32)

In [50]:
type(True)

bool

In [2]:
x={'a':5}
list(x)

['a']

In [5]:
import numpy as np
x=np.array([[[1,2,3],[2,3,4]],[[1,2,3],[2,3,4]]])

print(x)

[[[1 2 3]
  [2 3 4]]

 [[1 2 3]
  [2 3 4]]]
